The following notebook is built to analyze a clean tsv file, containing patents' data from Derwent database.
We are not allowed to share the content of the dataset (ex. graphics), but you can use the code to analyze your own files.

In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [2]:
data = pandas.read_csv('clean_patents_batteries.tsv', sep='\t', low_memory=False)
data = pandas.DataFrame(data)

In [ ]:
print(data.columns)

In [ ]:
print(set(data["Publication Country Code"]))  # CODICE PUBBLICAZIONE NAZIONE - solo un WO

In [ ]:
print(sorted(set(data["Publication Year"]))) # anni di pubblicazioni

In [ ]:
# classi tecnologiche brevetti generali
val = data["DWPI Class"].value_counts()
print(val[0:10])  # 10 classi tecnologiche più usate nei brevetti in 14 anni

In [ ]:
val[0:10].plot(kind="pie", subplots=True, legend=True, autopct='%.1f%%', labeldistance=None )
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
ragg_anno = data.groupby(data["Publication Year"])["DWPI Class"].count()
ragg_anno = ragg_anno.drop(2023, errors='ignore')  #eliminato anno 2023 date le poche osservazioni
print(ragg_anno)  # totali anno per anno
ragg_anno.plot()
plt.show()

In [ ]:
conteggio_per_anno = data.groupby(["Publication Year", "DWPI Class"])["DWPI Class"].count()
top_10_per_anno = conteggio_per_anno.groupby("Publication Year").nlargest(10).reset_index(level=0, drop=True)
print(top_10_per_anno)

In [10]:
### ANNO 2020
anno = 2020
df = pd.DataFrame(top_10_per_anno)  # trasformazione in data frame

In [ ]:
### PER ANNO 2020
print(df.loc[[anno]])  # solo i migliori 10 in un anno
df.loc[[anno]].plot(kind="pie", subplots=True, legend=True, autopct='%.1f%%', labeldistance=None )
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
### GRAFICO A BARRE - CICLO FOR ANNUALE PER CLASSE TECNOLOGIA BREVETTI
for i in range(2010,2024):          #ciclo per produrre barchar e numero di brevetti suddivisi per tipologia effettuati anno per anno
    print(df.loc[[i]])
    df.loc[[i]].plot(kind="bar")
    plt.show()

In [ ]:
### GRAFICO A TORTA -CICLO FOR ANNUALE PER CLASSE TECNOLOGICA BREVETTI
for i in range(2010,2024):          #ciclo per produrre pie e numero di brevetti suddivisi per tipologia effettuati anno per anno
    print(df.loc[[i]])
    df.loc[[i]].plot(kind="pie", subplots= True, legend= True,autopct='%.1f%%',labeldistance=None )
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

In [ ]:
enterprises = data['Ultimate Parent'].value_counts()
print(enterprises[0:21])

In [ ]:
conteggio_per_anno = data.groupby(["Publication Year", 'Ultimate Parent'])['Ultimate Parent'].count()
top_10_per_anno = conteggio_per_anno.groupby("Publication Year").nlargest(10).reset_index(level=0, drop=True)
print(top_10_per_anno)

In [ ]:
df = pd.DataFrame(top_10_per_anno)  # trasformazione in data frame
df = df.drop(2023, errors = 'ignore')

for i in range(2010,2023):          #ciclo per produrre piechar e numero di brevetti suddivisi per tipologia effettuati anno per anno
    print(df.loc[[i]])
    df.loc[[i]].plot(kind="pie", subplots=True, legend=False, autopct='%.1f%%', labeldistance=None )
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

In [ ]:
conteggio_per_anno = data.groupby(["Publication Year", 'Ultimate Parent'])['Ultimate Parent'].count()

# Trasforma la Series 'conteggio_per_anno' in un DataFrame e resetta l'indice
df = conteggio_per_anno.reset_index(name='conteggio')

# Filtra il DataFrame per includere solo gli anni tra il 2010 e il 2022
df_filtered = df[df["Publication Year"].between(2010, 2022)]

# Calcola il totale dei brevetti per società nel range di anni indicato
totali_per_societa = df_filtered.groupby("Ultimate Parent")["conteggio"].sum()

# Ordina le società per il totale dei brevetti e seleziona le prime 5
top_10_societa = totali_per_societa.sort_values(ascending=False).head(10)

# Calcola la media dei brevetti per società nel range di anni indicato
anni = len(df_filtered["Publication Year"].unique())
media_per_societa = top_10_societa / anni

# Genera il grafico a torta con la media per ogni società dei brevetti posseduti nel range di anni indicato
media_per_societa.plot(kind="pie", legend=False, autopct='%.1f%%')
plt.title("TUTTE LE CLASSI TECNOLOGICHE - Media per società dei brevetti posseduti nel range di anni 2010-2022 (Top 10)")
plt.ylabel('')  # Rimuovi l'etichetta dell'asse y
plt.legend(loc='upper right', bbox_to_anchor=(3, 1))
plt.show()

In [ ]:
# calcola il conteggio dei brevetti per ogni anno e società
conteggio_per_anno = data.groupby(["Publication Year", 'Ultimate Parent'])['Ultimate Parent'].count()

# trasforma la Series in un DataFrame e resetta l'indice
df = conteggio_per_anno.reset_index(name='conteggio')

# filtra il DataFrame per includere solo gli anni tra il 2010 e il 2022
df_filtered = df[df["Publication Year"].between(2010, 2022)]

# calcola il totale dei brevetti per società nel range di anni indicato
totali_per_societa = df_filtered.groupby("Ultimate Parent")["conteggio"].sum()

# ordina le società per il totale dei brevetti e seleziona le prime 5
top_10_societa = totali_per_societa.sort_values(ascending=False).head(10)

# crea una tabella pivot per avere le società come colonne
pivot_filtered = df_filtered[df_filtered['Ultimate Parent'].isin(top_10_societa.index)].pivot(index='Publication Year', columns='Ultimate Parent', values='conteggio')

# genera il grafico di serie storica per le top 5 società
pivot_filtered.plot()
plt.title("TUTTE LE CLASSI TECNOLOGICHE - Serie storiche per le top 10 società nel range di anni 2010-2022")
plt.ylabel('Numero di brevetti')
plt.xlabel('Anno')
plt.legend(loc='upper right', bbox_to_anchor=(3, 1))
plt.show()

In [ ]:
def ends_with_x21e(s):
    if isinstance(s, str):
        return bool(re.search(r'\bX21 E\b$', s))
    return False

# Filtra il DataFrame per includere solo le classi DWPI che terminano con "X21 E"
data_x21 = data[data["DWPI Class"].apply(ends_with_x21e)]

# Stampa il DataFrame filtrato
print(data_x21)

In [ ]:
conteggio_per_anno_x21 = data_x21.groupby(["Publication Year", 'Ultimate Parent'])['Ultimate Parent'].count()

# Trasforma la Series 'conteggio_per_anno' in un DataFrame e resetta l'indice
df_x21 = conteggio_per_anno_x21.reset_index(name='conteggio')

# Filtra il DataFrame per includere solo gli anni tra il 2010 e il 2022
df_filtered_x21 = df_x21[df_x21["Publication Year"].between(2010, 2022)]

# Calcola il totale dei brevetti per società nel range di anni indicato
totali_per_societa_x21 = df_filtered_x21.groupby("Ultimate Parent")["conteggio"].sum()

# Ordina le società per il totale dei brevetti e seleziona le prime 5
top_10_societa_x21 = totali_per_societa_x21.sort_values(ascending=False).head(10)

# Calcola la media dei brevetti per società nel range di anni indicato
anni_x21 = len(df_filtered_x21["Publication Year"].unique())
media_per_societa_x21 = top_10_societa_x21 / anni_x21

# Genera il grafico a torta con la media per ogni società dei brevetti posseduti nel range di anni indicato
media_per_societa_x21.plot(kind="pie", legend=False, autopct='%.1f%%')
plt.title("X21 E - Media per società dei brevetti posseduti nel range di anni 2010-2022 (Top 10)")
plt.ylabel('')  # Rimuovi l'etichetta dell'asse y
plt.legend(loc='upper right', bbox_to_anchor=(3, 1))
plt.show()

In [ ]:
# Crea una tabella pivot per avere le società come colonne
pivot_x21 = df_filtered_x21.pivot(index='Publication Year', columns='Ultimate Parent', values='conteggio')

# Filtra la tabella pivot per includere solo le top 10 società
top_10_societa_x21 = pivot_x21.sum().sort_values(ascending=False).head(10)
societa_selezionate_x21 = pivot_x21[top_10_societa_x21.index]

# Calcola le medie mobili su finestre temporali di 3 anni per le top 10 società
media_mobili_x21 = societa_selezionate_x21.rolling(window=3).mean()

# Genera il grafico di serie storiche con medie mobili per le top 10 società
media_mobili_x21.plot()
plt.title("X21 E - Serie storiche con medie mobili per le top 10 società nel range di anni 2010-2022")
plt.ylabel('Numero di brevetti')
plt.xlabel('Anno')
plt.legend(loc='upper right', bbox_to_anchor=(3, 1))
plt.show()

In [ ]:
#isoliamo solo i brevetti che hanno come classi tecnologiche o x21 o x22
cl_x22_x21 = data[data["DWPI Class"].str.contains('|'.join(['X21', 'X22']), case= False, na=False)]

conteggio_per_anno_x21_x22 = cl_x22_x21.groupby(["Publication Year", 'Ultimate Parent'])['Ultimate Parent'].count()

# Trasforma la Series 'conteggio_per_anno' in un DataFrame e resetta l'indice
df_x21_x22 = conteggio_per_anno_x21_x22.reset_index(name='conteggio')


pivot_x21_x22 = df_x21_x22.pivot(index='Publication Year', columns='Ultimate Parent', values='conteggio')
top_10_societa_x21_x22 = pivot_x21_x22.sum().sort_values(ascending=False).head()
societa_selezionate_x21_x22 = pivot_x21[top_10_societa_x21_x22.index]

# genera il grafico di serie storica per le top 5 società
societa_selezionate_x21_x22.plot()
plt.title("X21-X22 - Serie storiche per le top 5 società nel range di anni 2010-2022")
plt.ylabel('Numero di brevetti')
plt.xlabel('Anno')
plt.legend(loc='upper right')
plt.show()


In [ ]:
ragg_anno_x22_x21 = cl_x22_x21.groupby(data["Publication Year"])["DWPI Class"].count()
ragg_anno_x22_x21 = ragg_anno_x22_x21.drop(2023, errors='ignore')  #eliminato anno 2023 date le poche osservazioni
print(ragg_anno_x22_x21)  # totali anno per anno
plt.clf()
ragg_anno_x22_x21.plot()
plt.show()